Vincent Cohen

DSCI 550

2/6/26

# <center> Homework 1

## Load Data and fill missing values:

Note: When you find missing values, remove the flight record (i.e., entire row which has missing value(s)) in the analysis. When you see a negative delay number, it can be interpreted as an early departure and arrival, so it will not be considered as a delay (i.e., treat them as zeros).

In [14]:
import pandas as pd

data = pd.read_csv('flights.csv') # Load in data from file

data = data.dropna() # Remove rows with missing values

data.loc[data["dep_delay"] < 0, "dep_delay"] = 0 # Set negative departure delays to zero
data.loc[data["arr_delay"] < 0, "arr_delay"] = 0 # Set negative arrival delays to zero
print(data.shape)
print(data.head())

(157927, 16)
   year  month  day  dep_time  dep_delay  arr_time  arr_delay carrier tailnum  \
0  2013      1    1     517.0        2.0     830.0       11.0      UA  N14228   
1  2013      1    1     533.0        4.0     850.0       20.0      UA  N24211   
2  2013      1    1     542.0        2.0     923.0       33.0      AA  N619AA   
3  2013      1    1     554.0        0.0     812.0        0.0      DL  N668DN   
4  2013      1    1     554.0        0.0     740.0       12.0      UA  N39463   

   flight origin dest  air_time  distance  hour  minute  
0    1545    EWR  IAH     227.0      1400   5.0    17.0  
1    1714    LGA  IAH     227.0      1416   5.0    33.0  
2    1141    JFK  MIA     160.0      1089   5.0    42.0  
3     461    LGA  ATL     116.0       762   5.0    54.0  
4    1696    EWR  ORD     150.0       719   5.0    54.0  


## Question 1:

- How many flights were recorded for each month in the dataset? 
- And what is the average departure delay time for each month?

In [95]:
by_month = (
    data.groupby(by=["month"])
    .agg(
        flight_count = pd.NamedAgg(column="month", aggfunc="size"),
        avg_dep_delay = pd.NamedAgg(column="dep_delay", aggfunc="mean"),
    )
    .reset_index()
)
print(by_month.sort_values(by="avg_dep_delay", ascending=False))


    month  flight_count  avg_dep_delay
5       6         13353      19.570434
6       7         13686      19.222198
11     12         13076      14.752218
3       4         13442      13.322943
4       5         13506      12.333778
7       8         14011      12.102919
2       3         13481      12.063348
1       2         11369       9.075380
8       9         12784       8.920526
0       1         12585       8.776559
9      10         13700       7.309489
10     11         12934       7.067419


The month with the least flights was February with just 11,369 and the month with the most flights was August with 14,011. The highest monthly delay was an average of 19.6 minutes in June and the lowest was surprisingly November with an average of just 7.1 minutes. 

## Question 2:

- Which were the top 5 most common destinations from the 'JFK' airport, based on the frequency?

In [81]:
jfk_origin = (
    data[data["origin"] == "JFK"]
    .groupby(by=["dest"])
    .size()
    .reset_index(name="flight_count")
)
print(jfk_origin.sort_values(by="flight_count", ascending=False).head(5))

   dest  flight_count
14  LAX          7711
28  SFO          5687
16  MIA          3285
3   BOS          2389
29  SJU          2379


The top 5 most common destinations from JFK are:
1. LAX (7,711 times)
2. SFO (5,687 times)
3. MIA (3,285 times)
4. BOS (2,389 times)
5. SJU (2,379 times)

## Question 3: 

- Calculate the total distance traveled by each carrier. 
- Which carrier traveled the most distance overall?

In [ ]:
by_carrier = (
    data.groupby(by=["carrier"])
    .agg(
        flight_count = pd.NamedAgg(column="carrier", aggfunc="size"),
        total_distance = pd.NamedAgg(column="distance", aggfunc="sum"),
    )
    .reset_index()
)
print(by_carrier.sort_values(by="total_distance", ascending=False))

  carrier  total_flights  total_distance
3      UA          57782        88482811
2      DL          47658        58999610
0      AA          31947        42913762
4      US          19831        11121739
1      AS            709         1703018


UA (United) traveled the most distance and operated the most flights in the data set. Their 88,482,811 miles flown are almost 30 million miles more than second place Delta. 

## Question 4: 

- How much departure delay do I need to expect when I use AA in December? (based on average)

In [87]:
aa_in_december = (
    data[(data["carrier"] == "AA") & (data["month"] == 12)]
    .groupby(by=["carrier"])
    .agg(
        flight_count = pd.NamedAgg(column="carrier", aggfunc="size"),
        avg_dep_delay = pd.NamedAgg(column="dep_delay", aggfunc="mean"),
    )
)
print(aa_in_december)

         flight_count  avg_dep_delay
carrier                             
AA               2597      13.934155


Based on the 2,597 flights AA operated in December, you should expect a delay of ~14 minutes (~48 seconds faster than the average for December for all carriers). 

## Question 5: 

- Determine the top 5 busiest days (based on the number of flights). 
- Report the year, month, day, and the number of flights.

In [ ]:
data["flight_date"] = pd.to_datetime(data[["year", "month", "day"]]) # group by full date

by_day = (
    data.groupby(data["flight_date"].dt.normalize())
      .size()
      .rename("flight_count")
      .reset_index()
)
print(by_day.sort_values(by="flight_count", ascending=False).head(5))

    flight_date  flight_count
226  2013-08-15           486
234  2013-08-23           483
232  2013-08-21           483
227  2013-08-16           483
167  2013-06-17           483


The 5 busiest days in the dataset are as follows:
1. August 15, 2013: 486 flights
2. August 23, 2013: 483 flights
3. August 21, 2013: 483 flights
4. August 16, 2013: 483 flights
5. June 17, 2013: 483 flights